In [1]:
import numpy as np
import tensorflow as tf
from __future__ import division, print_function, unicode_literals

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Tensorflow and Deep Learning

In this lab assignment, first you will learn how to build and train a neural network that recognises handwritten digits, and then you will build LeNet-5 CNN architecture, which is widely used for handwritten digit recognition. At the end of this lab assignment, you will make AlexNet CNN architecture, which won the 2012 ImageNet ILSVRC challenge.

---
# 1. Dataset
In the first part of the assignment, we use the MNIST dataset, which is a set of 70,000 small images of digits handwritten by high school students and employees of the US Census Bureau. Each image is labeled with the digit it represents. There are 70,000 images, and each image has 784 features. This is because each image is 28×28=784 pixels, and each feature simply represents one pixel's intensity, from 0 (white) to 255 (black). The following figure shows a few images from the MNIST dataset to give you a feel for the complexity of the classification task.

<img src="figs/1-mnist.png" style="width: 300px;"/>

To begin the assignment, first, use `mnist_data.read_data_sets` and download images and labels. It return two lists, called `mnist.test` with 10K images+labels, and `mnist.train` with 60K images+labels.

In [2]:
# TODO: Replace <FILL IN> with appropriate code

from tensorflow.examples.tutorials.mnist import input_data as mnist_data

mnist = mnist_data.read_data_sets("train", one_hot=True, reshape=False)

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting train\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting train\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting train\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting train\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please 

---
# 2. A One-Layer Neural Network
<img src="figs/2-comic1.png" style="width: 500px;"/>

Let's start by building a one-layer neural network. Handwritten digits in the MNIST dataset are 28x28 pixel greyscale images. The simplest approach for classifying them is to use the 28x28=784 pixels as inputs for a **one-layer neural network**. Each neuron in the network does a weighted sum of all of its inputs, adds a bias and then feeds the result through some non-linear activation function. Here we design a one-layer neural network with 10 output neurons since we want to classify digits into 10 classes (0 to 9).
<img src="figs/3-one_layer.png" style="width: 400px;"/>


For a classification problem, an *activation function* that works well is **softmax**. Applying softmax on a vector is done by taking the exponential of each element and then normalising the vector.
<img src="figs/4-softmax.png" style="width: 300px;"/>

We can summarise the behaviour of this single layer of neurons into a simple formula using a *matrix multiply*. If we give input data into the network in *mini-batch* of 100 images, it produces 100 predictions as the output. We define the **weights matrix $W$** with 10 columns, in which each column indicates the weight of a one class (a single digit), from 0 to 9. Using the first column of $W$, we can compute the weighted sum of all the pixels of the first image. This sum corresponds to the first neuron that points to the number 0. Using the second column of $W$, we do the same for the second neuron (number 1) and so on until the 10th neuron. We can then repeat the operation for the remaining 99 images in the mini-batch. If we call $X$ the matrix containing our 100 images (each row corresponds to one digit), all the weighted sums for our 10 neurons, computed on 100 images are simply $X.W$. Each neuron must now add its bias. Since we have 10 neurons, we have 10 bias constants. We finally apply the **softmax activation function** and obtain the formula describing a one-layer neural network, applied to 100 images.
<img src="figs/5-xw.png" style="width: 600px;"/>
<img src="figs/6-softmax2.png" style="width: 500px;"/>

Then, we need to use the **cross-entropy** to measure how good the predictions are, i.e., the distance between what the network tells us and what we know to be the truth. The cross-entropy is a function of weights, biases, pixels of the training image and its known label. If we compute the partial derivatives of the cross-entropy relatively to all the weights and all the biases, we obtain a **gradient**, computed for a given image, label and present value of weights and biases. We can update weights and biases by a fraction of the gradient and do the same thing again using the next batch of training images.
<img src="figs/7-cross_entropy.png" style="width: 600px;"/>

### Define Variables and Placeholders
First we define TensorFlow **variables** and **placeholders**. *Variables* are all the parameters that you want the training algorithm to determine for you (e.g., weights and biases). *Placeholders* are parameters that will be filled with actual data during training (e.g., training images). The shape of the tensor holding the training images is [None, 28, 28, 1] which stands for:
  - 28, 28, 1: our images are 28x28 (784) pixels x 1 value per pixel (grayscale). The last number would be 3 for color images and is not really necessary here.
  - None: this dimension will be the number of images in the mini-batch. It will be known at training time.

We also need an additional placeholder for the training labels that will be provided alongside training images.

In [5]:
# TODO: Replace <FILL IN> with appropriate code

# neural network with 1 layer of 10 softmax neurons
#
# · · · · · · · · · ·       (input data, flattened pixels)       X [batch, 784] 
# \x/x\x/x\x/x\x/x\x/    -- fully connected layer (softmax)      W [784, 10]     b[10]
#   · · · · · · · ·                                              Y_hat [batch, 10]

# input X: 28x28 grayscale images, the first dimension (None) will index the images in the mini-batch
X = tf.placeholder(tf.float32, shape=[None, 28, 28, 1], name="input")

# correct answers will go here
Y = tf.placeholder(tf.float32, shape=[None, 10], name="labels")

# weights W[784, 10], 784 = 28 * 28
W = tf.Variable(tf.zeros([784, 10]), name="weights")

# biases b[10]
b = tf.Variable(tf.zeros([10]), name="biases")

### Build The Model
Now, we can make a **model** for a one-layer neural network. The formula is the one we explained before, i.e., $\hat{Y} = softmax(X . W + b)$. You can use the `tf.nn.softmax` and `tf.matmul` to build the model. Here, we need to use the `tf.reshape` to transform our 28x28 images into single vectors of 784 pixels.

In [6]:
# TODO: Replace <FILL IN> with appropriate code

# flatten the images into a single line of pixels
XX = tf.reshape(X, [-1, 784], name="flatten_inputs")

# The model
Y_hat = tf.nn.softmax(tf.add(tf.matmul(XX, W), b), name="predictions")

### Define The Cost Function
Now, we have model predictions $\hat{Y}$ and correct labels $Y$, so for each instance $i$ (image) we can compute the cross-entropy as the **cost function**: $cross\_entropy = -\sum(Y_i * log(\hat{Y}i))$. You can use `reduce_mean` to add all the components in a tensor.

In [7]:
# TODO: Replace <FILL IN> with appropriate code

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels= Y, logits = Y_hat))

### Traine the Model
Now, select the gradient descent optimiser `GradientDescentOptimizer` and ask it to minimise the cross-entropy cost. In this step, TensorFlow computes the partial derivatives of the cost function relatively to all the weights and all the biases (the gradient). The gradient is then used to update the weights and biases. Set the learning rate is $0.005$.

In [8]:
# TODO: Replace <FILL IN> with appropriate code

optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.005)
train_step = optimizer.minimize(cross_entropy)

### Execute the Model
It is time to run the training loop. All the TensorFlow instructions up to this point have been preparing a computation graph in memory but nothing has been computed yet. The computation requires actual data to be fed into the placeholders. This is supplied in the form of a Python dictionary, where the keys are the names of the placeholders. During the trainig print out the cost every 200 steps. Moreove, after training the model, print out the accurray of the model by testing it on the test data.

In [13]:
# TODO: Replace <FILL IN> with appropriate code

# init
init = tf.global_variables_initializer()

X_i, Y_train = mnist.train.images, mnist.train.labels
# infer the batch size for the input layer
X_train = np.reshape(X_i, (-1, 28, 28, 1))

X_test_i, Y_test = mnist.test.images, mnist.test.labels
# blah
X_test = np.reshape(X_test_i, (-1, 28, 28, 1))

pred = tf.equal(tf.argmax(Y_hat, 1), tf.argmax(Y, 1))
accu = tf.reduce_mean(tf.cast(pred, tf.float32))


n_epochs = 5000
with tf.Session() as sess:
    sess.run(init)
    for i in range(n_epochs):
        batch_x_i, batch_y = mnist.train.next_batch(100)
        batch_x = np.reshape(batch_x_i, (-1, 28, 28, 1))
        sess.run(train_step, feed_dict={X: batch_x, Y:batch_y})
        if i % 200 == 0:
            train_loss = sess.run(cross_entropy, feed_dict={X: X_train, Y: Y_train})
            print(f'Training loss step {i}: {train_loss}')
            test_accu = sess.run(accu, feed_dict={X: X_test, Y: Y_test})
            print(f'Test accuracy step {i}:  {test_accu*100}%')
            print("----------------------------------------------------")
    final_test_accu = sess.run(accu, feed_dict={X: X_test, Y: Y_test})
    print(f'Final test accuracy: {final_test_accu*100}%')

Training loss step 0: 2.302647829055786
Test accuracy step 0:  44.839999079704285%
----------------------------------------------------
Training loss step 200: 2.290116786956787
Test accuracy step 200:  60.69999933242798%
----------------------------------------------------
Training loss step 400: 2.2725372314453125
Test accuracy step 400:  49.84999895095825%
----------------------------------------------------
Training loss step 600: 2.2474844455718994
Test accuracy step 600:  41.20999872684479%
----------------------------------------------------
Training loss step 800: 2.2213900089263916
Test accuracy step 800:  45.68000137805939%
----------------------------------------------------
Training loss step 1000: 2.1942994594573975
Test accuracy step 1000:  53.07000279426575%
----------------------------------------------------
Training loss step 1200: 2.1632533073425293
Test accuracy step 1200:  55.22000193595886%
----------------------------------------------------
Training loss step 14

---
# 2. Add More Layers

<img src="figs/8-comic2.png" style="width: 500px;"/>

Now, let's improve the recognition accuracy by adding more layers to the neural network. The neurons in the second layer, instead of computing weighted sums of pixels will compute weighted sums of neuron outputs from the previous layer. We keep the softmax function as the activation function on the last layer, but on intermediate layers we will use the the **sigmoid** activation function. So, let's build a five-layer fully connected neural network with the following structure, and train the model with the trainging data and print out its accuracy on the test data.
<img src="figs/9-five_layer.png" style="width: 500px;"/>

In [14]:
# TODO: Replace <FILL IN> with appropriate code

# neural network with five layers
#
# · · · · · · · · · ·          (input data, flattened pixels)       X [batch, 784]   
# \x/x\x/x\x/x\x/x\x/       -- fully connected layer (sigmoid)      W1 [784, 200]      B1 [200]
#  · · · · · · · · ·                                                Y1_hat [batch, 200]
#   \x/x\x/x\x/x\x/         -- fully connected layer (sigmoid)      W2 [200, 100]      B2 [100]
#    · · · · · · ·                                                  Y2_hat [batch, 100]
#     \x/x\x/x\x/           -- fully connected layer (sigmoid)      W3 [100, 60]       B3 [60]
#      · · · · ·                                                    Y3_hat [batch, 60]
#       \x/x\x/             -- fully connected layer (sigmoid)      W4 [60, 30]        B4 [30]
#        · · ·                                                      Y4_hat [batch, 30]
#         \x/               -- fully connected layer (softmax)      W5 [30, 10]        B5 [10]
#          ·                                                        Y_hat [batch, 10]

# to reset the Tensorflow default graph
reset_graph()

########################################
# define variables and placeholders
########################################
X = tf.placeholder(tf.float32, [None, 28, 28, 1], name="input")
Y = tf.placeholder(tf.float32, shape=(None, 10), name="labels")

# five layers and their number of neurons, i.e., 200, 100, 60, 30, and 10

# tf.truncated_normal vs tf.random_normal:
# tf.truncated_normal defeat the saturation of functions
# like sigmoid, if the value is too big/small, the neuron stops learning
# but there it should not be the problem as it is mnist, anyways I go with truncated_normal
W1 = tf.Variable(tf.truncated_normal([784, 200], stddev=0.1), name="w1")
B1 = tf.zeros([200], name="b1")

W2 = tf.Variable(tf.truncated_normal([200, 100], stddev=0.1), name="w2")
B2 = tf.zeros([100], name="b2")

W3 = tf.Variable(tf.truncated_normal([100, 60], stddev=0.1), name="w3")
B3 = tf.zeros([60], name="b3")

W4 = tf.Variable(tf.truncated_normal([60, 30], stddev=0.1), name="w4")
B4 = tf.zeros([30], name="b4")

W5 = tf.Variable(tf.truncated_normal([30, 10], stddev=0.1), name="w5")
B5 = tf.zeros([10], name="b5")

########################################
# build the model
########################################
XX = tf.reshape(X, [-1, 784], name="flatten")

Y1_hat = tf.nn.sigmoid(tf.add(tf.matmul(XX, W1), B1), name="o1")
Y2_hat = tf.nn.sigmoid(tf.add(tf.matmul(Y1_hat, W2), B2), name="o2")
Y3_hat = tf.nn.sigmoid(tf.add(tf.matmul(Y2_hat, W3), B3), name="o3")
Y4_hat = tf.nn.sigmoid(tf.add(tf.matmul(Y3_hat, W4), B4), name="o4")
Y_hat = tf.nn.softmax(tf.add(tf.matmul(Y4_hat, W5), B5), name="O")

########################################
# define the cost function
########################################
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels= Y, logits = Y_hat))

########################################
# define the optimizer
########################################
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.005)
train_step = optimizer.minimize(cross_entropy)

########################################
# execute the model
########################################
# init
init = tf.global_variables_initializer()

X_i, Y_train = mnist.train.images, mnist.train.labels
# infer the batch size for the input layer
X_train = np.reshape(X_i, (-1, 28, 28, 1))

X_test_i, Y_test = mnist.test.images, mnist.test.labels
# blah
X_test = np.reshape(X_test_i, (-1, 28, 28, 1))

pred = tf.equal(tf.argmax(Y_hat, 1), tf.argmax(Y, 1))
accu = tf.reduce_mean(tf.cast(pred, tf.float32))


n_epochs = 5000
with tf.Session() as sess:
    sess.run(init)
    for i in range(n_epochs):
        batch_x_i, batch_y = mnist.train.next_batch(100)
        batch_x = np.reshape(batch_x_i, (-1, 28, 28, 1))
        sess.run(train_step, feed_dict={X: batch_x, Y:batch_y})
        if i % 200 == 0:
            train_loss = sess.run(cross_entropy, feed_dict={X: X_train, Y: Y_train})
            print(f'Training loss step {i}: {train_loss}')
            test_accu = sess.run(accu, feed_dict={X: X_test, Y: Y_test})
            print(f'Test accuracy step {i}:  {test_accu*100}%')
            print("----------------------------------------------------")
    final_test_accu = sess.run(accu, feed_dict={X: X_test, Y: Y_test})
    print(f'Final test accuracy: {final_test_accu*100}%')

Training loss step 0: 2.3024797439575195
Test accuracy step 0:  10.090000182390213%
----------------------------------------------------
Training loss step 200: 2.302452802658081
Test accuracy step 200:  10.090000182390213%
----------------------------------------------------
Training loss step 400: 2.3024299144744873
Test accuracy step 400:  10.090000182390213%
----------------------------------------------------
Training loss step 600: 2.302401065826416
Test accuracy step 600:  10.090000182390213%
----------------------------------------------------
Training loss step 800: 2.3024256229400635
Test accuracy step 800:  10.090000182390213%
----------------------------------------------------
Training loss step 1000: 2.302377939224243
Test accuracy step 1000:  10.090000182390213%
----------------------------------------------------
Training loss step 1200: 2.3023648262023926
Test accuracy step 1200:  10.090000182390213%
----------------------------------------------------
Training loss st

---
# 3. Special Care for Deep Networks
As layers were added, neural networks tended to converge with more difficulties. For example, the accuracy could stuck at 0.1. Here, we want to apply some updates to the network we built in the previous part to improve its performance. 

### ReLU Activation Function
<img src="figs/10-comic3.png" style="width: 500px;"/>
The sigmoid activation function is actually quite problematic in deep networks. It squashes all values between 0 and 1 and when you do so repeatedly, neuron outputs and their gradients can vanish entirely. An alternative activation function is **ReLU** that shows better performance compare to sigmoid. It looks like as below:
<img src="figs/11-relu.png" style="width: 300px;"/>

### A Better Optimizer
In very high dimensional spaces like here, **saddle points** are frequent. These are points that are not local minima, but where the gradient is nevertheless zero and the gradient descent optimizer stays stuck there. One possible solution to tackle this probelm is to use better optimizers, such as Adam optimizer `tf.train.AdamOptimizer`.

### Random Initialisations
When working with ReLUs, the best practice is to initialise bias values to small positive values, so that neurons operate in the non-zero range of the ReLU initially.

### Learning Rate
<img src="figs/12-comic4.png" style="width: 500px;"/>
With two, three or four intermediate layers, you can now get close to 98% accuracy, if you push the iterations to 5000 or beyond. But, the results are not very consistent, and the curves jump up and down by a whole percent. A good solution is to start fast and decay the learning rate exponentially from $0.005$ to $0.0001$ for example. In order to pass a different learning rate to the `AdamOptimizer` at each iteration, you will need to define a new placeholder and feed it a new value at each iteration through `feed_dict`. Here is the formula for exponential decay: $learning\_rate = lr\_min + (lr\_max - lr\_min) * e^{\frac{-i}{2000}}$, where $i$ is the iteration number.

### NaN?
In the network you built in the last section, you might see accuracy curve crashes and the console outputs NaN for the cross-entropy. It may happen, because you are attempting to compute a $log(0)$, which is indeed Not A Number (NaN). Remember that the cross-entropy involves a log, computed on the output of the softmax layer. Since softmax is essentially an exponential, which is never zero, we should be fine, but with 32 bit precision floating-point operations, exp(-100) is already a genuine zero. TensorFlow has a handy function that computes the softmax and the cross-entropy in a single step, implemented in a numerically stable way. To use it, you will need to separate the weighted sum plus bias on the last layer, before softmax is applied and then give it with the true values to the function `tf.nn.softmax_cross_entropy_with_logits`.

In the code below, apply the following changes and show their impact on the accuracy of the model on training data, as well as the test data:
* Replace the sigmoid activation function with ReLU
* Use the Adam optimizer
* Initialize weights with small random values between -0.2 and +0.2, and make sure biases are initialised with small positive values, for example 0.1
* Update the learning rate in different iterations. Start fast and decay the learning rate exponentially from $0.005$ to $0.0001$, i.e., 
```
max_learning_rate = 0.005
min_learning_rate = 0.0001
decay_speed = 2000.0
```
* Use `tf.nn.softmax_cross_entropy_with_logits` to prevent getting NaN in output.

In [21]:
# TODO: Replace <FILL IN> with appropriate code

# neural network with 5 layers
#
# · · · · · · · · · ·          (input data, flattened pixels)       X [batch, 784]   
# \x/x\x/x\x/x\x/x\x/       -- fully connected layer (sigmoid)      W1 [784, 200]      B1[200]
#  · · · · · · · · ·                                                Y1_hat [batch, 200]
#   \x/x\x/x\x/x\x/         -- fully connected layer (sigmoid)      W2 [200, 100]      B2[100]
#    · · · · · · ·                                                  Y2_hat [batch, 100]
#     \x/x\x/x\x/           -- fully connected layer (sigmoid)      W3 [100, 60]       B3[60]
#      · · · · ·                                                    Y3_hat [batch, 60]
#       \x/x\x/             -- fully connected layer (sigmoid)      W4 [60, 30]        B4[30]
#        · · ·                                                      Y4_hat [batch, 30]
#         \x/               -- fully connected layer (softmax)      W5 [30, 10]        B5[10]
#          ·                                                        Y_hat [batch, 10]

# to reset the Tensorflow default graph
import math
reset_graph()

########################################
# define variables and placeholders
########################################
X = tf.placeholder(tf.float32, [None, 28, 28, 1], name="input")
Y = tf.placeholder(tf.float32, shape=(None, 10), name="labels")

# variable learning rate
learning_rate = tf.placeholder(tf.float32, shape=[])

# five layers and their number of neurons, i.e., 200, 100, 60, 30, and 10
# when using RELUs, make sure biases are initialised with small positive values, for example 0.1
bias_initializer = tf.constant_initializer(value=0.1)

W1 = tf.Variable(tf.truncated_normal([784, 200], stddev=0.1), name="w1")
B1 = tf.get_variable("b1", shape=[200], initializer=bias_initializer)

W2 = tf.Variable(tf.truncated_normal([200, 100], stddev=0.1), name="w2")
B2 = tf.get_variable("b2", shape=[100], initializer=bias_initializer)

W3 = tf.Variable(tf.truncated_normal([100, 60], stddev=0.1), name="w3")
B3 = tf.get_variable("b3", shape=[60], initializer=bias_initializer)

W4 = tf.Variable(tf.truncated_normal([60, 30], stddev=0.1), name="w4")
B4 = tf.get_variable("b4", shape=[30], initializer=bias_initializer)

W5 = tf.Variable(tf.truncated_normal([30, 10], stddev=0.1), name="w5")
B5 = tf.get_variable("b5", shape=[10], initializer=bias_initializer)
########################################
# build the model
########################################
XX = tf.reshape(X, [-1, 784], name="flatten")

Y1_hat = tf.nn.relu(tf.add(tf.matmul(XX, W1), B1), name="o1")
Y2_hat = tf.nn.relu(tf.add(tf.matmul(Y1_hat, W2), B2), name="o2")
Y3_hat = tf.nn.relu(tf.add(tf.matmul(Y2_hat, W3), B3), name="o3")
Y4_hat = tf.nn.relu(tf.add(tf.matmul(Y3_hat, W4), B4), name="o4")
Y_hat = tf.nn.softmax(tf.add(tf.matmul(Y4_hat, W5), B5), name="O")

########################################
# defining the cost function
########################################
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=Y_hat)
cross_entropy = tf.reduce_mean(cross_entropy) * 100

########################################
# define the optimizer
########################################

max_lr = 0.005
min_lr = 0.0001
decay_speed = 2000.0
L = min_lr +  tf.train.exponential_decay(max_lr, learning_rate, decay_speed, 1/math.e)
optimizer = tf.train.AdamOptimizer(L)
train_step = optimizer.minimize(cross_entropy)

########################################
# execute the model
########################################
# init
init = tf.global_variables_initializer()

X_i, Y_train = mnist.train.images, mnist.train.labels
# infer the batch size for the input layer
X_train = np.reshape(X_i, (-1, 28, 28, 1))

X_test_i, Y_test = mnist.test.images, mnist.test.labels
# blah
X_test = np.reshape(X_test_i, (-1, 28, 28, 1))

pred = tf.equal(tf.argmax(Y_hat, 1), tf.argmax(Y, 1))
accu = tf.reduce_mean(tf.cast(pred, tf.float32))

n_epochs = 5000
with tf.Session() as sess:
    sess.run(init)
    for i in range(n_epochs):
        batch_x_i, batch_y = mnist.train.next_batch(100)
        batch_x = np.reshape(batch_x_i, (-1, 28, 28, 1))
        sess.run(train_step, feed_dict={X: batch_x, Y:batch_y, learning_rate:i})
        if i % 200 == 0:
            train_loss = sess.run(cross_entropy, feed_dict={X: X_train, Y: Y_train, learning_rate:i})
            print(f'Training loss step {i}: {train_loss}')
            test_accu = sess.run(accu, feed_dict={X: X_test, Y: Y_test, learning_rate:i})
            print(f'Test accuracy step {i}:  {test_accu*100}%')
            print("----------------------------------------------------")
    final_test_accu = sess.run(accu, feed_dict={X: X_test, Y: Y_test})
    print(f'Final test accuracy: {final_test_accu*100}%')

Training loss step 0: 229.86505126953125
Test accuracy step 0:  23.45999926328659%
----------------------------------------------------
Training loss step 200: 157.51206970214844
Test accuracy step 200:  88.88000249862671%
----------------------------------------------------
Training loss step 400: 155.39236450195312
Test accuracy step 400:  91.17000102996826%
----------------------------------------------------
Training loss step 600: 155.22195434570312
Test accuracy step 600:  90.85000157356262%
----------------------------------------------------
Training loss step 800: 153.0153045654297
Test accuracy step 800:  92.89000034332275%
----------------------------------------------------
Training loss step 1000: 152.28704833984375
Test accuracy step 1000:  93.54000091552734%
----------------------------------------------------
Training loss step 1200: 151.2333221435547
Test accuracy step 1200:  94.33000087738037%
----------------------------------------------------
Training loss step 140

---
# 4. Overfitting and Dropout
<img src="figs/13-comic5.png" style="width: 500px;"/>
You will have noticed that cross-entropy curves for test and training data start disconnecting after a couple thousand iterations. The learning algorithm works on training data only and optimises the training cross-entropy accordingly. It never sees test data so it is not surprising that after a while its work no longer has an effect on the test cross-entropy which stops dropping and sometimes even bounces back up. 
<img src="figs/14-overfit.png" style="width: 500px;"/>
This disconnect is usually labeled **overfitting** and when you see it, you can try to apply a regularisation technique called **dropout**. In dropout, at each training iteration, you drop random neurons from the network. You choose a probability `pkeep` for a neuron to be kept, usually between 50% and 75%, and then at each iteration of the training loop, you randomly remove neurons with all their weights and biases. Different neurons will be dropped at each iteration. When testing the performance of your network of course you put all the neurons back (`pkeep = 1`).
<img src="figs/15-dropout.png" style="width: 500px;"/>
TensorFlow offers a dropout function to be used on the outputs of a layer of neurons. It randomly zeroes-out some of the outputs and boosts the remaining ones by `1 / pkeep`. You can add dropout after each intermediate layer in the network now. 

In the following code, use the dropout between each layer during the training, and set the probability `pkeep` once to $50%$ and another time to $75%$ and compare their results.

In [27]:
# TODO: Replace <FILL IN> with appropriate code
import math
# neural network with 5 layers
#
# · · · · · · · · · ·          (input data, flattened pixels)       X [batch, 784]   
# \x/x\x/x\x/x\x/x\x/       -- fully connected layer (sigmoid)      W1 [784, 200]      B1[200]
#  · · · · · · · · ·                                                Y1_hat [batch, 200]
#   \x/x\x/x\x/x\x/         -- fully connected layer (sigmoid)      W2 [200, 100]      B2[100]
#    · · · · · · ·                                                  Y2_hat [batch, 100]
#     \x/x\x/x\x/           -- fully connected layer (sigmoid)      W3 [100, 60]       B3[60]
#      · · · · ·                                                    Y3_hat [batch, 60]
#       \x/x\x/             -- fully connected layer (sigmoid)      W4 [60, 30]        B4[30]
#        · · ·                                                      Y4_hat [batch, 30]
#         \x/               -- fully connected layer (softmax)      W5 [30, 10]        B5[10]
#          ·                                                        Y_hat [batch, 10]

# to reset the Tensorflow default graph
reset_graph()

########################################
# define variables and placeholders
########################################
X = tf.placeholder(tf.float32, [None, 28, 28, 1], name="input")
Y = tf.placeholder(tf.float32, shape=(None, 10), name="labels")

# variable learning rate
learning_rate = tf.placeholder(tf.float32, shape=[])

# probability of keeping a node during dropout = 1.0 at test time (no dropout) and 0.75 at training time
pkeep = tf.placeholder(tf.float32, shape=[])

# five layers and their number of neurons, i.e., 200, 100, 60, 30, and 10
bias_initializer = tf.constant_initializer(value=0.1)

W1 = tf.Variable(tf.truncated_normal([784, 200], stddev=0.1), name="w1")
B1 = tf.get_variable("b1", shape=[200], initializer=bias_initializer)

W2 = tf.Variable(tf.truncated_normal([200, 100], stddev=0.1), name="w2")
B2 = tf.get_variable("b2", shape=[100], initializer=bias_initializer)

W3 = tf.Variable(tf.truncated_normal([100, 60], stddev=0.1), name="w3")
B3 = tf.get_variable("b3", shape=[60], initializer=bias_initializer)

W4 = tf.Variable(tf.truncated_normal([60, 30], stddev=0.1), name="w4")
B4 = tf.get_variable("b4", shape=[30], initializer=bias_initializer)

W5 = tf.Variable(tf.truncated_normal([30, 10], stddev=0.1), name="w5")
B5 = tf.get_variable("b5", shape=[10], initializer=bias_initializer)

########################################
# build the model
########################################
XX = tf.reshape(X, [-1, 784], name="flatten")

Y1_hat = tf.nn.relu(tf.add(tf.matmul(XX, W1), B1), name="o1")
Y1_hat_dropout = tf.nn.dropout(Y1_hat, keep_prob=pkeep, name="d1")
Y2_hat_dropout = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(Y1_hat_dropout, W2), B2)), keep_prob=pkeep, name="d2")
Y3_hat_dropout = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(Y2_hat_dropout, W3), B3)), keep_prob=pkeep, name="d3")
Y4_hat_dropout = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(Y3_hat_dropout, W4), B4)), keep_prob=pkeep, name="d4")
Y_hat = tf.nn.softmax(tf.add(tf.matmul(Y4_hat_dropout, W5), B5), name="d5")

########################################
# define the cost function
########################################
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=Y_hat)
cross_entropy = tf.reduce_mean(cross_entropy) * 100

########################################
# define the optimizer
########################################
max_lr = 0.005
min_lr = 0.0001
decay_speed = 2000.0
L = min_lr +  tf.train.exponential_decay(max_lr, learning_rate, decay_speed, 1/math.e)
optimizer = tf.train.AdamOptimizer(L)
train_step = optimizer.minimize(cross_entropy)

########################################
# execute the model
########################################
init = tf.global_variables_initializer()

X_i, Y_train = mnist.train.images, mnist.train.labels
# infer the batch size for the input layer
X_train = np.reshape(X_i, (-1, 28, 28, 1))

X_test_i, Y_test = mnist.test.images, mnist.test.labels
# blah
X_test = np.reshape(X_test_i, (-1, 28, 28, 1))

pred = tf.equal(tf.argmax(Y_hat, 1), tf.argmax(Y, 1))
accu = tf.reduce_mean(tf.cast(pred, tf.float32))

n_epochs = 5000
for dropout in [0.5, 0.75]:
    print(f"Dropout = {dropout}")
    print('Start:----------------------------------------')
    with tf.Session() as sess:
        sess.run(init)
        for i in range(n_epochs):
            batch_x_i, batch_y = mnist.train.next_batch(100)
            batch_x = np.reshape(batch_x_i, (-1, 28, 28, 1))
            sess.run(train_step, feed_dict={X: batch_x, Y:batch_y, learning_rate:i, pkeep: dropout})
            if i % 200 == 0:
                train_loss = sess.run(cross_entropy, feed_dict={X: X_train, Y: Y_train, learning_rate:i, pkeep: dropout})
                print(f'Training loss step {i}: {train_loss}')
                test_accu = sess.run(accu, feed_dict={X: X_test, Y: Y_test, learning_rate:i, pkeep: 1.0})
                print(f'Test accuracy step {i}:  {test_accu*100}%')
                print("----------------------------------------------------")
        final_test_accu = sess.run(accu, feed_dict={X: X_test, Y: Y_test, pkeep: 1.0})
        print(f'Final test accuracy: {final_test_accu*100}%')
        print('End:----------------------------------------')

Dropout = 0.5
Start:----------------------------------------
Training loss step 0: 230.20973205566406
Test accuracy step 0:  9.589999914169312%
----------------------------------------------------
Training loss step 200: 186.58352661132812
Test accuracy step 200:  67.22999811172485%
----------------------------------------------------
Training loss step 400: 177.51956176757812
Test accuracy step 400:  83.41000080108643%
----------------------------------------------------
Training loss step 600: 170.2686004638672
Test accuracy step 600:  87.3199999332428%
----------------------------------------------------
Training loss step 800: 168.50840759277344
Test accuracy step 800:  88.77000212669373%
----------------------------------------------------
Training loss step 1000: 166.31585693359375
Test accuracy step 1000:  90.17000198364258%
----------------------------------------------------
Training loss step 1200: 164.35638427734375
Test accuracy step 1200:  91.01999998092651%
--------------

#### Increasing dropout beyond a certain threshold results in the model not being able to fit properly. Intuitively, a higher dropout rate would result in a higher variance to some of the layers, which also degrades training. Dropout reduces model capacity. Reduce the capacity too much will lead to results. One possible issue for that sudden increase may be solved by lowering the learning rate (used) or using higher momentum.

---
# 6. Convolutional Network
<img src="figs/16-comic6.png" style="width: 500px;"/>
In the previous sections, all pixels of images flattened into a single vector, which was a really bad idea. Handwritten digits are made of shapes and we discarded the shape information when we flattened the pixels. However, we can use **convolutional neural networks (CNN)** to take advantage of shape information. CNNs apply *a series of filters* to the raw pixel data of an image to extract and learn higher-level features, which the model can then use for classification. CNNs contains three components:
  - **Convolutional layers**: apply a specified number of convolution filters to the image. For each subregion, the layer performs a set of mathematical operations to produce a single value in the output feature map. Convolutional layers then typically apply a ReLU activation function to the output to introduce nonlinearities into the model.
  - **Pooling layers**: downsample the image data extracted by the convolutional layers to reduce the dimensionality of the feature map in order to decrease processing time. A commonly used pooling algorithm is max pooling, which extracts subregions of the feature map (e.g., 2x2-pixel tiles), keeps their maximum value, and discards all other values.
  - **Dense (fully connected) layers**: perform classification on the features extracted by the convolutional layers and downsampled by the pooling layers. In a dense layer, every node in the layer is connected to every node in the preceding layer.
  
Typically, a CNN is composed of a *stack of **convolutional modules*** that perform feature extraction. Each *module* consists of a *convolutional layer* followed by a *pooling layer*. The last convolutional module is followed by one or more dense layers that perform classification. The final dense layer in a CNN contains a single neuron for each target class in the model, with a softmax activation function to generate a value between 0-1 for each neuron. We can interpret the softmax values for a given image as relative measurements of how likely it is that the image falls into each target class.

Now, let us build a convolutional network for handwritten digit recognition. In this assignment, we will use the architecture shown in the following figure that has three convolutional layers, one fully-connected layer, and one softmax layer. Notice that the second and third convolutional layers have a stride of two that explains why they bring the number of output values down from 28x28 to 14x14 and then 7x7. A convolutional layer requires a weights tensor like `[4, 4, 3, 2]`, in which the first two numbers define the size of a filter (map), the third number shows the *depth* of the filter that is the number of *input channel*, and the last number shows the number of *output channel*. The output channel defines the number of times that we repeat the same thing with a different set of weights in one layer. In our implementation, we assume the output depth of first three convolutional layers, are 4, 8, 12, and the size of fully connected layer is 200.
<img src="figs/17-arch1.png" style="width: 600px;"/>

Convolutional layers can be implemented in TensorFlow using the `tf.nn.conv2d` function, which performs the scanning of the input image in both directions using the supplied weights. This is only the weighted sum part of the neuron. You still need to add a bias and feed the result through an activation function. The padding strategy that works here is to copy pixels from the sides of the image. All digits are on a uniform background so this just extends the background and should not add any unwanted shapes.

In [28]:
# TODO: Replace <FILL IN> with appropriate code

# · · · · · · · · · ·      (input data, 1-deep)               X [batch, 28, 28, 1]
# @ @ @ @ @ @ @ @ @ @   -- conv. layer 5x5x1=>4 stride 1      W1 [5, 5, 1, 4]        B1 [4]
# ∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶                                         Y1_hat [batch, 28, 28, 4]
#   @ @ @ @ @ @ @ @     -- conv. layer 5x5x4=>8 stride 2      W2 [5, 5, 4, 8]        B2 [8]
#   ∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶                                           Y2_hat [batch, 14, 14, 8]
#     @ @ @ @ @ @       -- conv. layer 4x4x8=>12 stride 2     W3 [4, 4, 8, 12]       B3 [12]
#     ∶∶∶∶∶∶∶∶∶∶∶                                             Y3_hat [batch, 7, 7, 12] => reshaped to YY [batch, 7*7*12]
#      \x/x\x\x/        -- fully connected layer (relu)       W4 [7*7*12, 200]       B4 [200]
#       · · · ·                                               Y4_hat [batch, 200]
#       \x/x\x/         -- fully connected layer (softmax)    W5 [200, 10]           B5 [10]
#        · · ·                                                Y_hat [batch, 10]

# to reset the Tensorflow default graph
reset_graph()

########################################
# define variables and placeholders
########################################
X = tf.placeholder(tf.float32, [None, 28, 28, 1], name="input")
Y = tf.placeholder(tf.float32, shape=(None, 10), name="labels")

# variable learning rate
learning_rate = tf.placeholder(tf.float32, shape=[])

# three convolutional layers with their channel counts, and a fully connected layer 
# (the last layer has 10 softmax neurons)
# the output depth of first three convolutional layers, are 4, 8, 12, and the size of fully connected
# layer is 200
bias_initializer = tf.constant_initializer(value=0.1)

W1 = tf.Variable(tf.truncated_normal([5,5,1,4], stddev=0.1), name="w1")
B1 = tf.get_variable("b1", shape=[4], initializer=bias_initializer, trainable=True)

W2 = tf.Variable(tf.truncated_normal([5,5,4,8], stddev=0.1), name="w2")
B2 = tf.get_variable("b2", shape=[8], initializer=bias_initializer, trainable=True)

W3 = tf.Variable(tf.truncated_normal([4,4,8,12], stddev=0.1), name="w3")
B3 = tf.get_variable("b3", shape=[12], initializer=bias_initializer, trainable=True)

W4 = tf.Variable(tf.truncated_normal([7*7*12, 200], stddev=0.1), name="w4")
B4 = tf.get_variable("b4", shape=[200], initializer=bias_initializer, trainable=True)

W5 = tf.Variable(tf.truncated_normal([200, 10], stddev=0.1), name="w5")
B5 = tf.get_variable("b5", shape=[10], initializer=bias_initializer, trainable=True)

########################################
# build the model
########################################
stride = 1  # output is 28x28
Y1_hat = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, stride, stride, 1], padding = 'SAME') + B1) # use tf.nn.conv2d

stride = 2  # output is 14x14
Y2_hat = tf.nn.relu(tf.nn.conv2d(Y1_hat, W2, strides=[1, stride, stride, 1], padding = 'SAME') + B2)

stride = 2  # output is 7x7
Y3_hat = tf.nn.relu(tf.nn.conv2d(Y2_hat, W3, strides=[1, stride, stride, 1], padding = 'SAME') + B3)

# reshape the output from the third convolution for the fully connected layer
YY_hat = tf.reshape(Y3_hat, [-1, 7*7*12])
Y4_hat = tf.nn.relu(tf.matmul(YY_hat, W4) + B4)
Y_logit = tf.matmul(Y4_hat, W5) + B5
Y_hat = tf.nn.softmax(Y_logit)

########################################
# define the cost function
########################################
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Y_logit, labels=Y)
cross_entropy = tf.reduce_mean(cross_entropy) * 100

########################################
# define the optmizer
########################################
max_lr = 0.005
min_lr = 0.0001
decay_speed = 2000.0
L = min_lr +  tf.train.exponential_decay(max_lr, learning_rate, decay_speed, 1/math.e)
optimizer = tf.train.AdamOptimizer(L)
train_step = optimizer.minimize(cross_entropy)

########################################
# execute the model
########################################
init = tf.global_variables_initializer()

X_i, Y_train = mnist.train.images, mnist.train.labels
# infer the batch size for the input layer
X_train = np.reshape(X_i, (-1, 28, 28, 1))

X_test_i, Y_test = mnist.test.images, mnist.test.labels
# blah
X_test = np.reshape(X_test_i, (-1, 28, 28, 1))

pred = tf.equal(tf.argmax(Y_hat, 1), tf.argmax(Y, 1))
accu = tf.reduce_mean(tf.cast(pred, tf.float32))

n_epochs = 5000
with tf.Session() as sess:
    sess.run(init)
    for i in range(n_epochs):
        batch_x_i, batch_y = mnist.train.next_batch(100)
        batch_x = np.reshape(batch_x_i, (-1, 28, 28, 1))
        sess.run(train_step, feed_dict={X: batch_x, Y:batch_y, learning_rate:i})
        if i % 200 == 0:
            train_loss = sess.run(cross_entropy, feed_dict={X: X_train, Y: Y_train, learning_rate:i})
            print(f'Training loss step {i}: {train_loss}')
            test_accu = sess.run(accu, feed_dict={X: X_test, Y: Y_test, learning_rate:i})
            print(f'Test accuracy step {i}:  {test_accu*100}%')
            print("----------------------------------------------------")
    final_test_accu = sess.run(accu, feed_dict={X: X_test, Y: Y_test})
    print(f'Final test accuracy: {final_test_accu*100}%')

Training loss step 0: 247.94369506835938
Test accuracy step 0:  13.699999451637268%
----------------------------------------------------
Training loss step 200: 14.466534614562988
Test accuracy step 200:  95.87000012397766%
----------------------------------------------------
Training loss step 400: 7.919217109680176
Test accuracy step 400:  97.33999967575073%
----------------------------------------------------
Training loss step 600: 5.770322322845459
Test accuracy step 600:  98.03000092506409%
----------------------------------------------------
Training loss step 800: 4.866929054260254
Test accuracy step 800:  98.15000295639038%
----------------------------------------------------
Training loss step 1000: 3.9105918407440186
Test accuracy step 1000:  98.35000038146973%
----------------------------------------------------
Training loss step 1200: 2.907217264175415
Test accuracy step 1200:  98.50000143051147%
----------------------------------------------------
Training loss step 1400

# 7. Improve The Performance
A good approach to sizing your neural networks is to implement a network that is a little too constrained, then give it a bit more degrees of freedom and add dropout to make sure it is not overfitting. This ends up with a fairly optimal network for your problem. In the above model, we set the output channel to 4 in the first convolutional layer, which means that we repeat the same filter shape (but with different weights) four times. If we assume that those filters evolve during training into shape recognisers, you can intuitively see that this might not be enough for our problem. Handwritten digits are made from more than 4 elemental shapes. So let us bump up the filter sizes a little, and also increase the number of filters in our convolutional layers from 4, 8, 12 to 6, 12, 24 and then add dropout on the fully-connected layer. The following figure shows the new architecture you should build. Please complete the following code based on the given architecture and dropout technique.
<img src="figs/18-arch2.png" style="width: 600px;"/>

In [32]:
# TODO: Replace <FILL IN> with appropriate code

# · · · · · · · · · ·    (input data, 1-deep)                 X [batch, 28, 28, 1]
# @ @ @ @ @ @ @ @ @ @ -- conv. layer 6x6x1=>6 stride 1        W1 [5, 5, 1, 6]        B1 [6]
# ∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶                                         Y1_hat [batch, 28, 28, 6]
#   @ @ @ @ @ @ @ @   -- conv. layer 5x5x6=>12 stride 2       W2 [5, 5, 6, 12]        B2 [12]
#   ∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶                                           Y2_hat [batch, 14, 14, 12]
#     @ @ @ @ @ @     -- conv. layer 4x4x12=>24 stride 2      W3 [4, 4, 12, 24]       B3 [24]
#     ∶∶∶∶∶∶∶∶∶∶∶                                             Y3_hat [batch, 7, 7, 24] => reshaped to YY [batch, 7*7*24]
#      \x/x\x\x/ ✞    -- fully connected layer (relu+dropout) W4 [7*7*24, 200]       B4 [200]
#       · · · ·                                               Y4_hat [batch, 200]
#       \x/x\x/       -- fully connected layer (softmax)      W5 [200, 10]           B5 [10]
#        · · ·                                                Y_hat [batch, 10]

# to reset the Tensorflow default graph
reset_graph()

########################################
# define variables and placeholders
########################################
X = tf.placeholder(tf.float32, [None, 28, 28, 1], name="input")
Y = tf.placeholder(tf.float32, shape=(None, 10), name="labels")

# variable learning rate
learning_rate = tf.placeholder(tf.float32, shape=[])
pkeep = tf.placeholder(tf.float32, shape=[])
# three convolutional layers with their channel counts, and a fully connected layer 
# (the last layer has 10 softmax neurons)
# the output depth of first three convolutional layers, are 4, 8, 12, and the size of fully connected
# layer is 200
bias_initializer = tf.constant_initializer(value=0.1)

W1 = tf.Variable(tf.truncated_normal([6, 6, 1, 6], stddev=0.1), name="w1")
B1 = tf.get_variable("b1", shape=[6], initializer=bias_initializer, trainable=True)

W2 = tf.Variable(tf.truncated_normal([5, 5, 6, 12], stddev=0.1), name="w2")
B2 = tf.get_variable("b2", shape=[12], initializer=bias_initializer, trainable=True)

W3 = tf.Variable(tf.truncated_normal([4, 4, 12, 24], stddev=0.1), name="w3")
B3 = tf.get_variable("b3", shape=[24], initializer=bias_initializer, trainable=True)

W4 = tf.Variable(tf.truncated_normal([7 * 7 * 24, 200], stddev=0.1), name="w4")
B4 = tf.get_variable("b4", shape=[200], initializer=bias_initializer, trainable=True)

W5 = tf.Variable(tf.truncated_normal([200, 10], stddev=0.1), name="w5")
B5 = tf.get_variable("b5", shape=[10], initializer=bias_initializer, trainable=True)

########################################
# build the model
########################################
stride = 1  # output is 28x28
Y1_hat = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, stride, stride, 1], padding = 'SAME') + B1) # use tf.nn.conv2d

stride = 2  # output is 14x14
Y2_hat = tf.nn.relu(tf.nn.conv2d(Y1_hat, W2, strides=[1, stride, stride, 1], padding = 'SAME') + B2)

stride = 2  # output is 7x7
Y3_hat = tf.nn.relu(tf.nn.conv2d(Y2_hat, W3, strides=[1, stride, stride, 1], padding = 'SAME') + B3)

# reshape the output from the third convolution for the fully connected layer
YY_hat = tf.reshape(Y3_hat, [-1, 7 * 7 * 24])
Y4_hat = tf.nn.relu(tf.matmul(YY_hat, W4) + B4)
Y_logit = tf.matmul(Y4_hat, W5) + B5
Y_hat = tf.nn.softmax(Y_logit)

########################################
# define the cost function
########################################
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Y_logit, labels=Y)
cross_entropy = tf.reduce_mean(cross_entropy) * 100

########################################
# define the optmizer
########################################
max_lr = 0.005
min_lr = 0.0001
decay_speed = 2000.0
L = min_lr +  tf.train.exponential_decay(max_lr, learning_rate, decay_speed, 1/math.e)
optimizer = tf.train.AdamOptimizer(L)
train_step = optimizer.minimize(cross_entropy)

########################################
# execute the model
########################################
init = tf.global_variables_initializer()

X_i, Y_train = mnist.train.images, mnist.train.labels
# infer the batch size for the input layer
X_train = np.reshape(X_i, (-1, 28, 28, 1))

X_test_i, Y_test = mnist.test.images, mnist.test.labels
# blah
X_test = np.reshape(X_test_i, (-1, 28, 28, 1))

pred = tf.equal(tf.argmax(Y_hat, 1), tf.argmax(Y, 1))
accu = tf.reduce_mean(tf.cast(pred, tf.float32))

n_epochs = 5000
with tf.Session() as sess:
    sess.run(init)
    for i in range(n_epochs):
        batch_x_i, batch_y = mnist.train.next_batch(100)
        batch_x = np.reshape(batch_x_i, (-1, 28, 28, 1))
        sess.run(train_step, feed_dict={X: batch_x, Y:batch_y, learning_rate:i, pkeep:0.75})
        if i % 200 == 0:
            train_loss = sess.run(cross_entropy, feed_dict={X: X_train, Y: Y_train, learning_rate:i, pkeep:0.75})
            print(f'Training loss step {i}: {train_loss}')
            test_accu = sess.run(accu, feed_dict={X: X_test, Y: Y_test, learning_rate:i, pkeep:1.0})
            print(f'Test accuracy step {i}:  {test_accu*100}%')
            print("----------------------------------------------------")
    final_test_accu = sess.run(accu, feed_dict={X: X_test, Y: Y_test, pkeep:1.0})
    print(f'Final test accuracy: {final_test_accu*100}%')

Training loss step 0: 253.3010711669922
Test accuracy step 0:  15.080000460147858%
----------------------------------------------------
Training loss step 200: 9.72043514251709
Test accuracy step 200:  97.14999794960022%
----------------------------------------------------
Training loss step 400: 6.123346328735352
Test accuracy step 400:  97.99000024795532%
----------------------------------------------------
Training loss step 600: 6.013984203338623
Test accuracy step 600:  98.03000092506409%
----------------------------------------------------
Training loss step 800: 4.260636329650879
Test accuracy step 800:  98.53000044822693%
----------------------------------------------------
Training loss step 1000: 3.0984911918640137
Test accuracy step 1000:  98.69999885559082%
----------------------------------------------------
Training loss step 1200: 3.5178375244140625
Test accuracy step 1200:  98.58999848365784%
----------------------------------------------------
Training loss step 1400: 

---
# 8. Tensorflow Layers Module
The TensorFlow **layers** `tf.layers` module provides a high-level API that makes it easy to construct a neural network. It provides methods that facilitate: (i) the creation of dense (fully connected) layers and convolutional layers, (ii) adding activation functions, and (iii) applying dropout regularization. In this section use the module `tf.layers` to build the network you made in section 7.

In [130]:
# TODO: Replace <FILL IN> with appropriate code

# to reset the Tensorflow default graph
reset_graph()

X = tf.placeholder(tf.float32, [None, 28, 28, 1], name="input")
Y = tf.placeholder(tf.float32, shape=(None, 10), name="labels")

# variable learning rate
learning_rate = tf.placeholder(tf.float32, shape=[])
pkeep = tf.placeholder(tf.float32, shape=[])
bi = tf.constant_initializer(value=0.1)

# Layer 1 6x6x1
conv1 = tf.layers.conv2d(inputs = X, filters = 6, kernel_size = (6,6), 
                         strides = (1, 1), padding = 'SAME', 
                         activation = tf.nn.relu)
# Layer2 5x5x6
conv2 = tf.layers.conv2d(inputs = conv1, filters = 12, kernel_size = (5,5), 
                         strides = (2, 2), padding = 'SAME', 
                         activation = tf.nn.relu)
# Layer 3 4x4x12
conv3 = tf.layers.conv2d(inputs = conv2, filters = 24, kernel_size = (4,4), 
                         strides = (2, 2), padding = 'SAME', 
                         activation = tf.nn.relu)
flat = tf.reshape(conv3, shape=[-1, 7 * 7 * 24])

# fully-connected layer (relu)
dense = tf.layers.dense(inputs=flat, units=200, use_bias = True, 
                        activation = tf.nn.relu)
# dropout
dropout = tf.layers.dropout(inputs=dense, rate=pkeep)

# output layer (softmax)
output = tf.layers.dense(inputs=dropout, units=10, use_bias = True, 
                         activation = tf.nn.softmax)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=Y)
cross_entropy = tf.reduce_mean(cross_entropy) * 100

########################################
# define the optmizer
########################################
max_lr = 0.005
min_lr = 0.0001
decay_speed = 2000.0
L = min_lr +  tf.train.exponential_decay(max_lr, learning_rate, decay_speed, 1/math.e)
optimizer = tf.train.AdamOptimizer(L)
train_step = optimizer.minimize(cross_entropy)

########################################
# execute the model this time we use 0.75 dropout as we have 0.5 above
########################################
init = tf.global_variables_initializer()

X_i, Y_train = mnist.train.images, mnist.train.labels
# infer the batch size for the input layer
X_train = np.reshape(X_i, (-1, 28, 28, 1))

X_test_i, Y_test = mnist.test.images, mnist.test.labels
# blah
X_test = np.reshape(X_test_i, (-1, 28, 28, 1))

pred = tf.equal(tf.argmax(output, 1), tf.argmax(Y, 1))
accu = tf.reduce_mean(tf.cast(pred, tf.float32))

n_epochs = 5000
with tf.Session() as sess:
    sess.run(init)
    for i in range(n_epochs):
        batch_x_i, batch_y = mnist.train.next_batch(100)
        batch_x = np.reshape(batch_x_i, (-1, 28, 28, 1))
        sess.run(train_step, feed_dict={X: batch_x, Y:batch_y, learning_rate:i, pkeep:0.75})
        if i % 200 == 0:
            train_loss = sess.run(cross_entropy, feed_dict={X: X_train, Y: Y_train, learning_rate:i, pkeep:0.75})
            print(f'Training loss step {i}: {train_loss}')
            test_accu = sess.run(accu, feed_dict={X: X_test, Y: Y_test, learning_rate:i, pkeep:1.0})
            print(f'Test accuracy step {i}:  {test_accu*100}%')
            print("----------------------------------------------------")
    final_test_accu = sess.run(accu, feed_dict={X: X_test, Y: Y_test, pkeep:1.0})
    print(f'Final test accuracy: {final_test_accu*100}%')

Training loss step 0: 229.14039611816406
Test accuracy step 0:  16.699999570846558%
----------------------------------------------------
Training loss step 200: 153.55581665039062
Test accuracy step 200:  92.66999959945679%
----------------------------------------------------
Training loss step 400: 154.82798767089844
Test accuracy step 400:  91.86999797821045%
----------------------------------------------------
Training loss step 600: 151.62705993652344
Test accuracy step 600:  94.70000267028809%
----------------------------------------------------
Training loss step 800: 151.65907287597656
Test accuracy step 800:  94.9999988079071%
----------------------------------------------------
Training loss step 1000: 150.62574768066406
Test accuracy step 1000:  95.92000246047974%
----------------------------------------------------
Training loss step 1200: 150.83763122558594
Test accuracy step 1200:  95.45000195503235%
----------------------------------------------------
Training loss step 1

---
# 9. Keras
Keras is a high-level API to build and train deep learning models. It's used for fast prototyping, advanced research, and production. `tf.keras` is TensorFlow's implementation of the Keras API specification. To work with Keras, you need to import `tf.keras` as part of your TensorFlow program setup.
```
import tensorflow as tf
from tensorflow.keras import layers
```
#### Build a model
In Keras, you assemble **layers** to build a model, i.e., a graph of layers. The most common type of model is a stack of layers: the `tf.keras.Sequential` model. For example, the following code builds a simple, fully-connected network (i.e., multi-layer perceptron):
```
model = tf.keras.Sequential()
# adds a densely-connected layer with 64 units to the model:
model.add(layers.Dense(64, activation='relu'))
# add another
model.add(layers.Dense(64, activation='relu'))
# add a softmax layer with 10 output units:
model.add(layers.Dense(10, activation='softmax'))
```
There are many `tf.keras.layers` available with some common constructor parameters:
* `activation`: set the activation function for the layer, which is specified by the name of a built-in function or as a callable object.
* `kernel_initializer` and `bias_initializer`: the initialization schemes that create the layer's weights (weight and bias).
* `kernel_regularizer` and `bias_regularizer`: the regularization schemes that apply the layer's weights (weight and bias), such as L1 or L2 regularization.

#### Train and evaluate
After you construct a model, you can configure its learning process by calling the `compile` method:
```
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
```
The method `tf.keras.Model.compile` takes three important arguments:
* `optimizer`: it specifies the training procedure, e.g., `tf.train.AdamOptimizer` and `tf.train.GradientDescentOptimizer`.
* `loss`: the cost function to minimize during optimization, e.g., mean square error (mse), categorical_crossentropy, and binary_crossentropy.
* `metrics`: used to monitor training, e.g., `accuracy`.

The next step after confiuring the model is to train it by calling the `model.fit` method and giving it training data as its input. After training the model you can call `tf.keras.Model.evaluate` and `tf.keras.Model.predict` methods to evaluate the inference-mode loss and metrics for the data provided or predict the output of the last layer in inference for the data provided, respectively.

You can read more about Keras [here](https://www.tensorflow.org/guide/keras).

In this task, please use Keras to rebuild the network you made in section 7.

In [59]:
print(tf.__version__)

1.7.0


In [69]:
# TODO: Replace <FILL IN> with appropriate code
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Conv2D, Flatten, Dropout
# to reset the Tensorflow default graph
reset_graph()

model = Sequential()

model.add(Conv2D(6, (6, 6), strides=(1, 1), padding="same", 
                        activation="relu", name="conv1", input_shape = (28, 28, 1)))

model.add(Conv2D(12, (5, 5), strides=(2, 2), padding="same", 
                              activation="relu", name="conv2"))

model.add(Conv2D(24, (4, 4), strides=(2, 2), padding="same", 
                              activation="relu", name="conv3"))

model.add(Flatten(name="flatten"))

model.add(Dense(200, activation="relu", name="dense1"))

model.add(Dropout(0.75))

model.add(Dense(10, activation="softmax", name="dense2"))

model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [70]:
X_i.shape

(55000, 28, 28, 1)

In [71]:
# 55k training set, we cannot run this notebook on that hops stuff (cannot install keras there)
# as a consequence, we need to run little epochs
model.fit(mnist.train.images, mnist.train.labels, batch_size=200, epochs=5)

Epoch 1/5
55000/55000 [==============================] - 32s 577us/step - loss: 0.5596 - acc: 0.8232
Epoch 2/5
55000/55000 [==============================] - 31s 566us/step - loss: 0.2072 - acc: 0.9405
Epoch 3/5
55000/55000 [==============================] - 31s 564us/step - loss: 0.1459 - acc: 0.9584
Epoch 4/5
55000/55000 [==============================] - 31s 564us/step - loss: 0.1163 - acc: 0.9665
Epoch 5/5
55000/55000 [==============================] - 33s 592us/step - loss: 0.0998 - acc: 0.9707


In [72]:
score = model.evaluate(mnist.test.images, mnist.test.labels, verbose=0)
print('\nTest loss:', score[0])
print('Test accuracy:', score[1])


Test loss: 0.04346492445904369
Test accuracy: 0.9868


---
# 10. Implement LeNet-5
In this section, you should implement **LeNet-5** either using Tensorflow or Keras. Please take a look at its [paper](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf) before starting to implement it.
The LeNet-5 architecture is perhaps the most widely known CNN architecture. It was created by Yann LeCun in 1998 and widely used for handwritten digit recognition (MNIST). It is composed of the layers shown in the following table.
<img src="figs/19-letnet5.png" style="width: 600px;"/>
There are a few extra details to be noted:
* MNIST images are 28×28 pixels, but they are zero-padded to 32×32 pixels and normalized before being fed to the network. The rest of the network does not use any padding, which is why the size keeps shrinking as the image progresses through the network.
* The average pooling layers are slightly more complex than usual: each neuron computes the mean of its inputs, then multiplies the result by a learnable coefficient and adds a learnable bias term, then finally applies the activation function.
* Most neurons in layer C3 maps are connected to neurons in only three or four S2 maps (instead of all six S2 maps). See table 1 in the [paper](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf) for details.
* The output layer is a bit special: instead of computing the dot product of the inputs and the weight vector, each neuron outputs the square of the Euclidian distance between its input vector and its weight vector. Each output measures how much the image belongs to a particular digit class. The cross-entropy cost function is now preferred, as it penalizes bad predictions much more, producing larger gradients and thus converging faster.

In [93]:
# TODO: Build the LetNet-5 model, and test it on MNIST
import tensorflow as tf
from keras import backend as K
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Conv2D, Flatten, Dropout, AveragePooling2D
# to reset the Tensorflow default graph
tf.keras.backend.clear_session()
reset_graph()
# We use keras
# we believe the activation function should be ReLU, but img shows tanh...

LeNet5 = Sequential()
#C1
LeNet5.add(Conv2D(filters = 6, kernel_size = 5, strides = 1, activation = 'tanh', 
                 input_shape = (32, 32, 1)))
LeNet5.add(AveragePooling2D(pool_size = 2, strides = 2))

#C2
LeNet5.add(Conv2D(filters = 16, kernel_size = 5, strides = 1, activation = 'tanh',
                 input_shape = (14, 14, 6)))
LeNet5.add(AveragePooling2D(pool_size = 2, strides = 2))

#Flatten
LeNet5.add(Flatten())

#Fully connected
LeNet5.add(Dense(units = 120, activation = 'tanh'))

#Fully connected layer 2
LeNet5.add(Dense(units = 84, activation = 'tanh'))

#Output Layer, cross-entropy cost function is now preferred
LeNet5.add(Dense(units = 10, activation = 'softmax'))
LeNet5.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [94]:
LeNet5.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 6)         156       
_________________________________________________________________
average_pooling2d_1 (Average (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
average_pooling2d_2 (Average (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               48120     
_________________________________________________________________
dense_2 (Dense)              (None, 84)                10164     
__________

In [95]:
X_train = np.pad(mnist.train.images, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_test = np.pad(mnist.test.images, ((0,0),(2,2),(2,2),(0,0)), 'constant')

In [96]:
LeNet5.fit(X_train, mnist.train.labels, batch_size=200, epochs=5)

Epoch 1/5
55000/55000 [==============================] - 18s 330us/step - loss: 0.4111 - acc: 0.88441s - los
Epoch 2/5
55000/55000 [==============================] - 17s 317us/step - loss: 0.1562 - acc: 0.9533
Epoch 3/5
55000/55000 [==============================] - 18s 329us/step - loss: 0.1042 - acc: 0.9692
Epoch 4/5
55000/55000 [==============================] - 18s 322us/step - loss: 0.0783 - acc: 0.9764
Epoch 5/5
55000/55000 [==============================] - 17s 317us/step - loss: 0.0599 - acc: 0.9818


In [98]:
lscore = LeNet5.evaluate(X_test, mnist.test.labels, verbose=0)
print('\nTest loss:', lscore[0])
print('Test accuracy:', lscore[1])


Test loss: 0.0643268140753731
Test accuracy: 0.9799


In [ ]:
# Tensorflow edition, optional, if I got a time


---
# 11. Implement AlexNet
In the last section, you should implement **AlexNet** either using Tensorflow or Keras. Again, please take a look at its [paper](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) before start to implement it.
The AlexNet CNN architecture won the [ImageNet ILSVRC challenge](http://www.image-net.org/challenges/LSVRC/2012/) in 2012 by a large margin. It was developed by Alex Krizhevsky, Ilya Sutskever, and Geoffrey Hinton. It is quite similar to LeNet-5, only much larger and deeper, and it was the first to stack convolutional layers directly on top of each other, instead of stacking a pooling layer on top of each convolutional layer. The following table presents this architecture.
<img src="figs/20-alexnet.png" style="width: 600px;"/>
To train the model, we need a big dataset, however, in this assignment you are going to to assign the pretrained weights to your model, using `tf.Variable.assign`. You can download the pretrained weights from [bvlc_alexnet.npy](https://www.cs.toronto.edu/~guerzhoy/tf_alexnet/bvlc_alexnet.npy). This file is a NumPy array file created by the python. After you read this file, you will receive a python dictionary with a <key, value> pair for each layer. Each key is one of the layers names, e.g., `conv1`, and each value is a list of two values: (1) weights, and (2) biases of that layer. Part of the function to load the weights and biases to your model is given, and you need to complete it.

Here is what you see if you read and print the shape of each layer from the file:
```
weight_dic = np.load("bvlc_alexnet.npy", encoding="bytes").item()
for layer in weights_dic:
    print("-" * 20)
    print(layer)
    for wb in weights_dic[layer]:
        print(wb.shape)

#--------------------
# fc8
# (4096, 1000) # weights
# (1000,) # bias
#--------------------
# fc7
# (4096, 4096) # weights
# (4096,) # bias
#--------------------
# fc6
# (9216, 4096) # weights
# (4096,) # bias
#--------------------
# conv5
# (3, 3, 192, 256) # weights
# (256,) # bias
#--------------------
# conv4
# (3, 3, 192, 384) # weights
# (384,) # bias
#--------------------
# conv3
# (3, 3, 256, 384) # weights
# (384,) # bias
#--------------------
# conv2
# (5, 5, 48, 256) # weights
# (256,) # bias
#--------------------
# conv1
# (11, 11, 3, 96) # weights
# (96,) # bias
```


In [3]:
# TODO: Replace <FILL IN> with appropriate code

# to reset the Tensorflow default graph
reset_graph()

# X = tf.placeholder(tf.float32, [None, 224 * 224 * 3], name="input")
# Y = tf.placeholder(tf.float32, shape=(None, 1000), name="labels")

# # variable learning rate
# learning_rate = tf.placeholder(tf.float32, shape=[])
# pkeep = tf.placeholder(tf.float32, shape=[])

# why the heckkkkkkkkkkkkkkkkkkkkkkkkkkkkkk!!!!!!!!!!!!!!!!!!!
# # you can fine-tuning the lrn adjusting the depth and alpha, I didnt do it before, neither now
# def AlexNet(_X, _weights, _biases):
#     _X = tf.reshape(_X, shape=[-1, 224, 224, 3])
#     # conv1
#     conv1 = tf.nn.conv2d(_X, weights["wc1"], strides=[1, 4, 4, 1], padding="SAME", name="conv1")
#     conv1 = tf.nn.bias_add(conv1, biases["bc1"])
#     conv1 = tf.nn.relu(conv1)
#     conv1 = tf.nn.lrn(conv1)
#     conv1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="VALID")

#     # conv2
#     conv2 = tf.nn.conv2d(conv1, weights["wc2"], strides=[1, 1, 1, 1], padding="SAME", name="conv2")
#     conv2 = tf.nn.bias_add(conv2, biases["bc2"])
#     conv2 = tf.nn.relu(conv2)
#     conv2 = tf.nn.lrn(conv2)
#     conv2 = tf.nn.max_pool(conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="VALID")

#     # conv3
#     conv3 = tf.nn.conv2d(conv2, weights["wc3"], strides=[1, 1, 1, 1], padding="SAME", name="conv3")
#     conv3 = tf.nn.bias_add(conv3, biases["bc3"])
#     conv3 = tf.nn.relu(conv3)

#     # conv4
#     conv4 = tf.nn.conv2d(conv3, weights["wc4"], strides=[1, 1, 1, 1], padding="SAME", name="conv4")
#     conv4 = tf.nn.bias_add(conv4, biases["bc4"])
#     conv4 = tf.nn.relu(conv4)
    
#     #conv5
#     conv5 = tf.nn.conv2d(conv4, weights["wc5"], strides=[1, 1, 1, 1], padding="SAME", name="conv5")
#     conv5 = tf.nn.bias_add(conv5, biases["bc5"])
#     conv5 = tf.nn.relu(conv5)
#     conv5 = tf.nn.max_pool(conv5, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="VALID")

#     #flatten
#     flatten = tf.reshape(conv5, [-1, weights['wf1'].get_shape().as_list()[0]])
    
#     #fc1: there is a dropout 50% from the img above
#     fc1 = tf.nn.bias_add(tf.matmul(flatten, weights["wf1"]), biases["bf1"], name="fc1")
#     fc1 = tf.nn.relu(fc1)
#     fc1 = tf.nn.dropout(fc1, keep_prob=0.5)
    
#     #fc2
#     fc2 = tf.nn.bias_add(tf.matmul(fc1, weights["wf2"]), biases["bf2"], name="fc2")
#     fc2 = tf.nn.relu(fc2)
#     fc2 = tf.nn.dropout(fc2, keep_prob=0.5)
    
#     #fc3
#     fc3 = tf.nn.bias_add(tf.matmul(fc2, weights["wf3"]), biases["bf3"], name="fc3")
    
#     return fc3

# # all weights tuple
# weights = {
#     "wc1": tf.Variable(tf.truncated_normal([11, 11, 3, 96], stddev=0.1), name="wc1"),
#     "wc2": tf.Variable(tf.truncated_normal([5, 5, 96, 256], stddev=0.1), name="wc2"),
#     "wc3": tf.Variable(tf.truncated_normal([3, 3, 256, 384], stddev=0.1), name="wc3"),
#     "wc4": tf.Variable(tf.truncated_normal([3, 3, 384, 384], stddev=0.1), name="wc4"),
#     "wc5": tf.Variable(tf.truncated_normal([3, 3, 384, 256], stddev=0.1), name="wc5"),
#     "wf1": tf.Variable(tf.truncated_normal([28*28*256, 4096], stddev=0.1), name="wf1"),
#     "wf2": tf.Variable(tf.truncated_normal([4096, 4096], stddev=0.1), name="wf2"),
#     "wf3": tf.Variable(tf.truncated_normal([4096, 1000], stddev=0.1), name="wf3")
# }

# # all biases, make it random, dont wanna reason it.... 
# biases = {
#     "bc1": tf.Variable(tf.random_normal([96]), name="bc1"),
#     "bc2": tf.Variable(tf.random_normal([256]), name="bc2"),
#     "bc3": tf.Variable(tf.random_normal([384]), name="bc3"),
#     "bc4": tf.Variable(tf.random_normal([384]), name="bc4"),
#     "bc5": tf.Variable(tf.random_normal([256]), name="bc5"),
#     "bf1": tf.Variable(tf.random_normal([4096]), name="bf1"),
#     "bf2": tf.Variable(tf.random_normal([4096]), name="bf2"),
#     "bf3": tf.Variable(tf.random_normal([1000]), name="bf3")
# }

# Some trials-----------------------------------------
# define layers

def maxPool(x, kheight, kwidth, stridex, stridey, name, padding = "SAME"):
    return tf.nn.max_pool(x, ksize = [1, kheight, kwidth, 1],
                          strides = [1, stridex, stridey, 1], padding = padding, name = name)

def dropout(x, keep_prob, name = None):
    return tf.nn.dropout(x, keep_prob, name)

def LRN(x, name = None):
    return tf.nn.lrn(x, name = name)

    
def fc(x, inputf, outputf, name, relu = True):
    with tf.variable_scope(name) as scope:
        w = tf.get_variable("w", shape = [inputf, outputf], dtype = "float")
        b = tf.get_variable("b", [outputf], dtype = "float")
        output = tf.nn.xw_plus_b(x, w, b, name = scope.name)
        if relu:
            return tf.nn.relu(output)
        else:
            return output

        
def conv(x, kheight, kwidth, stridex, stridey,
              filters, name, poolgroups = 1, padding = "SAME"): 
    channel = int(x.get_shape()[-1])
#   debug this shit convolve func
    conv = lambda i, j: tf.nn.conv2d(i, j, strides = [1, stridey, stridex, 1], padding = padding)
    with tf.variable_scope(name) as scope:
        w = tf.get_variable("w", shape = [kheight, kwidth, channel/poolgroups, filters])
        b = tf.get_variable("b", shape = [filters])
        
        x_split = tf.split(value = x, num_or_size_splits = poolgroups, axis = 3)
        w_split = tf.split(value = w, num_or_size_splits = poolgroups, axis = 3)
        
        #after group2 split pooling, we have 2 fmaps
        featureMaps = [conv(p, q) for p, q in zip(x_split, w_split)]
        #now merge them
        merged = tf.concat(axis = 3, values = featureMaps)
    
        addBias = tf.nn.bias_add(merged, b)
        # need to reshape here..
#         merged.shape()
        output = tf.nn.relu(tf.reshape(addBias, tf.shape(merged)), name = scope.name)
        return output



# layers done-----------------------------------------------------

# build the model class
class AlexNet(object):
     def __init__(self, _x, prob):
        self.X = _x
        self.PROB = prob 
        self.build()
    
     def build(self):
        #conv1
        conv1 = conv(self.X, 11, 11, 4, 4, 96, name = "conv1", padding = "VALID")
        lrn1 = LRN(conv1, "norm1")
        pool1 = maxPool(lrn1, 3, 3, 2, 2, name = "pool1", padding = "VALID")
        #conv2
        conv2 = conv(pool1, 5, 5, 1, 1, 256, name = "conv2", poolgroups = 2)
        lrn2 = LRN(conv2, "lrn2")
        pool2 = maxPool(lrn2, 3, 3, 2, 2, name = "pool2", padding = "VALID")
        #conv3
        conv3 = conv(pool2, 3, 3, 1, 1, 384, name = "conv3")
        #conv4
        conv4 = conv(conv3, 3, 3, 1, 1, 384, name = "conv4", poolgroups = 2)
        #conv5
        conv5 = conv(conv4, 3, 3, 1, 1, 256, name = "conv5", poolgroups = 2)
        pool5 = maxPool(conv5, 3, 3, 2, 2, name = "pool5", padding = "VALID")

        flatten = tf.reshape(pool5, [-1, 256 * 6 * 6])
        fc1 = fc(flatten, 256 * 6 * 6, 4096, "fc6")
        dropout1 = dropout(fc1, self.PROB)

        fc2 = fc(dropout1, 4096, 4096, "fc7")
        dropout2 = dropout(fc2, self.PROB)

        self.fc3 = fc(dropout2, 4096, 1000, "fc8", relu = False)

     # load inital weights and biases to the model
     # you serious? I freakin...
     # 
     def load_initial_weights(self, session):
         # load the weights into memory
         weights_dic = np.load('bvlc_alexnet.npy', encoding='bytes').item()
 
         # loop over all layer names stored in the weights dict
         for layer in weights_dict:
             with tf.variable_scope(layer, reuse=True):
                 # loop over list of weights/biases and assign them to their corresponding tf variable
                 for wb in weights_dict[layer]:
                     # biases
                     if len(wb.shape) == 1:
                        bias = tf.get_variable('b', trainable = False)
                        session.run(bias.assign(wb))
                     # weights
                     else:
                        weight = tf.get_variable('w', trainable = False)
                        session.run(weight.assign(wb))
                 

#### Test the model
After building the AlexNet model, you can test it on different images and present the accuracy of the model. To do so, first you need to use **OpenCV** library to make the images ready to give as input to the model. OpenCV is a library used for image processing. Below you can see how to read an image file and pre-process it using OpenCV to give it to the model. However, you need to complete the code and test the accuracy of your model. The teset images (shown below) are available in the `test_images` folder.
<table width="100%">
<tr>
<td><img src="test_images/test_image1.jpg" style="width:200px;"></td>
<td><p align="center"><img src="test_images/test_image2.jpg" style="width:200px;"></td>
<td align="right"><img src="test_images/test_image3.jpg" style="width:200px;"></td>
</tr>

In [4]:
keep_prob = tf.placeholder(tf.float32)
# man da heck the dim is 227*227 not 224*224 why dont mention it in your img
X = tf.placeholder(tf.float32, [1, 227, 227, 3], name="input")
model = AlexNet(X, keep_prob)
out = model.fc3
result = tf.nn.softmax(out)

In [5]:
# result.shape
out.shape

TensorShape([Dimension(1), Dimension(1000)])

In [6]:
# TODO: Replace <FILL IN> with appropriate code
# test the AlexNet model on the given images

import cv2
import os
import matplotlib

inputshape = tf.placeholder(tf.float32, [1, 227, 227, 3])

#get list of all images
current_dir = os.getcwd()
image_path = os.path.join(current_dir, 'test_images')
img_files = [os.path.join(image_path, f) for f in os.listdir(image_path) if f.endswith('.jpg')]

#load all images
imgs = []
for f in img_files:
    imgs.append(cv2.imread(f))

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    
    # loop over all images
    for i, image in enumerate(imgs):
        # convert image to float32 and resize to (227x227)
        img = cv2.resize(image.astype(np.float32), (227, 227))
        
        # subtract the ImageNet mean
        # Mean subtraction per channel was used to center the data around zero mean for each channel (R, G, B).
        # This typically helps the network to learn faster since gradients act uniformly for each channel.
        imagenet_mean = np.array([104., 117., 124.], dtype=np.float32)
        img -= imagenet_mean
        
        # reshape as needed to feed into model
        img = img.reshape((1, 227, 227, 3))
        
        p = sess.run(result, feed_dict={X: img, keep_prob: 1})
        

# matplot didnot show the image, we therefore use cv to plot the image with prob,
        max_prob = np.max(p) 
        cv2.imshow('Image{} with test probability {}'.format(os.path.basename(img_files[i]), 
                                                        max_prob), image)
        cv2.imwrite('Image{} with test probability {}.png'.format(os.path.basename(img_files[i]), 
                                                        max_prob),img)
        cv2.waitKey(5000)
#         print('Image{} with test probability {}'.format(os.path.basename(img_files[i]), 
#                                                         max_prob)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


we cannot display the plot from the matplotlib somehow, therefore we just display the imgs with cv2 imshow every 2k milliseconds. We output the test probability as the title of the image and the result is not so good.

Imagetest_image1.jpg with test probability 0.0012396840611472726

Imagetest_image2.jpg with test probability 0.0012339357053861022

Imagetest_image3.jpg with test probability 0.0012296420754864812